IMPORTAÇÕES DE BIBLIOTECAS

In [10]:
import pandas as pd

LENDO O CSV DAS TABELAS DO BRASILEIRÃO DE 2003 A 2O22

In [11]:
df_br = pd.read_csv('Dados/campeonato-brasileiro-full.csv')

CRIANDO E ORGANIZANDO UM DATAFRAME APENAS COM OS JOGOS DO SÃO PAULO EM 2017 

In [12]:
df_br['data'] = pd.to_datetime(df_br['data'], dayfirst=True)

def determinar_vencedor(win):
    if win['mandante_Placar'] > win['visitante_Placar']:
        return win['mandante']
    elif win['mandante_Placar'] < win['visitante_Placar']:
        return win['visitante']
    else:
        return 'Empate'

def pontuação_sp(row):
    pontos_sao_paulo = 0
    if row['resultado'] == 'Sao Paulo':
        pontos_sao_paulo += 3
    elif row['resultado'] == 'Empate':
        pontos_sao_paulo += 1
    return pontos_sao_paulo

colunas_filtros = ['rodata', 'mandante', 'visitante', 'mandante_Placar', 'visitante_Placar', 'resultado', 'pontos']

CRIANDO UM DATA FRAME COM OS JOGOS PRÉ HERNANES

In [13]:
jogos_antes_hernanes = df_br[(df_br['data'].dt.year == 2017) & (df_br['rodata'] <= 16)].copy()  
jogos_sao_paulo_antes_hernanes = jogos_antes_hernanes[(jogos_antes_hernanes['mandante'] == 'Sao Paulo') | (jogos_antes_hernanes['visitante'] == 'Sao Paulo')].copy()  
jogos_sao_paulo_antes_hernanes['resultado'] = jogos_sao_paulo_antes_hernanes.apply(determinar_vencedor, axis=1)
jogos_sao_paulo_antes_hernanes['pontos'] = jogos_sao_paulo_antes_hernanes.apply(pontuação_sp, axis=1)
jogos_sao_paulo_antes_hernanes = jogos_sao_paulo_antes_hernanes.loc[:, colunas_filtros].copy()  

jogos_sao_paulo_antes_hernanes['soma_pontos_antes_hernanes'] = jogos_sao_paulo_antes_hernanes['pontos'].cumsum()

jogos_sao_paulo_antes_hernanes['total_pontos_possiveis_antes_hernanes'] = (jogos_sao_paulo_antes_hernanes['rodata'] - 1) * 3

print("Jogos Antes de Hernanes:")
print(jogos_sao_paulo_antes_hernanes.to_string(index=False))





Jogos Antes de Hernanes:
 rodata     mandante   visitante  mandante_Placar  visitante_Placar    resultado  pontos  soma_pontos_antes_hernanes  total_pontos_possiveis_antes_hernanes
      1     Cruzeiro   Sao Paulo                1                 0     Cruzeiro       0                           0                                      0
      2    Sao Paulo        Avai                2                 0    Sao Paulo       3                           3                                      3
      3    Sao Paulo   Palmeiras                2                 0    Sao Paulo       3                           6                                      6
      4  Ponte Preta   Sao Paulo                1                 0  Ponte Preta       0                           6                                      9
      5    Sao Paulo     Vitoria                2                 0    Sao Paulo       3                           9                                     12
      6  Corinthians   Sao Paulo       

CRIANDO UM DATAFRAME COM OS JOGOS PÓS HERNANES

In [14]:
jogos_depois_hernanes = df_br[(df_br['data'].dt.year == 2017) & (df_br['rodata'] >= 17)].copy()  
jogos_sao_paulo_depois_hernanes = jogos_depois_hernanes[(jogos_depois_hernanes['mandante'] == 'Sao Paulo') | (jogos_depois_hernanes['visitante'] == 'Sao Paulo')].copy()  
jogos_sao_paulo_depois_hernanes['resultado'] = jogos_sao_paulo_depois_hernanes.apply(determinar_vencedor, axis=1)
jogos_sao_paulo_depois_hernanes['pontos'] = jogos_sao_paulo_depois_hernanes.apply(pontuação_sp, axis=1)
jogos_sao_paulo_depois_hernanes = jogos_sao_paulo_depois_hernanes.loc[:, colunas_filtros].copy()  

jogos_sao_paulo_depois_hernanes['soma_pontos_depois_hernanes'] = jogos_sao_paulo_depois_hernanes['pontos'].cumsum()

jogos_sao_paulo_depois_hernanes['total_pontos_possiveis_depois_hernanes'] = (jogos_sao_paulo_depois_hernanes['rodata'] - 16) * 3

print("\nJogos Depois de Hernanes:")
print(jogos_sao_paulo_depois_hernanes.to_string(index=False))



Jogos Depois de Hernanes:
 rodata    mandante    visitante  mandante_Placar  visitante_Placar   resultado  pontos  soma_pontos_depois_hernanes  total_pontos_possiveis_depois_hernanes
     17 Botafogo-RJ    Sao Paulo                3                 4   Sao Paulo       3                            3                                       3
     18   Sao Paulo     Coritiba                1                 2    Coritiba       0                            3                                       6
     19       Bahia    Sao Paulo                2                 1       Bahia       0                            3                                       9
     20   Sao Paulo     Cruzeiro                3                 2   Sao Paulo       3                            6                                      12
     21        Avai    Sao Paulo                1                 1      Empate       1                            7                                      15
     22   Palmeiras    Sao Paul